In [1]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

In [2]:
df = pd.read_csv("../raw_data/final_to_model_df.csv")
df.columns

Index(['ID', 'City', 'Date', 'Season', 'MatchNumber', 'Team1', 'Team2',
       'Venue', 'TossWinner', 'TossDecision', 'WinningTeam', 'innings_total',
       'TeamA_batting_average', 'TeamB_batting_average', 'TeamA_innings_total',
       'TeamB_innings_total'],
      dtype='object')

In [3]:
df = df.drop(columns=['Date', 'Season'])

In [4]:
df.dropna(axis=0, inplace=True)

In [5]:
df.head()

,ID,City,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,innings_total,TeamA_batting_average,TeamB_batting_average,TeamA_innings_total,TeamB_innings_total
0,1312200,ahmedabad,final,rajasthan royals,gujarat titans,"narendra modi stadium, ahmedabad",rajasthan royals,bat,gujarat titans,130,155.397906,166.437500,130.0,133.0
1,1312199,ahmedabad,qualifier 2,royal challengers bangalore,rajasthan royals,"narendra modi stadium, ahmedabad",rajasthan royals,field,rajasthan royals,157,155.707965,155.397906,157.0,161.0
2,1312198,kolkata,eliminator,royal challengers bangalore,lucknow super giants,eden gardens,lucknow super giants,field,royal challengers bangalore,207,155.707965,169.866667,207.0,193.0
3,1312197,kolkata,qualifier 1,rajasthan royals,gujarat titans,eden gardens,gujarat titans,field,gujarat titans,188,155.397906,166.437500,188.0,191.0
4,1304116,mumbai,70,sunrisers hyderabad,punjab kings,wankhede stadium,sunrisers hyderabad,bat,punjab kings,157,155.546053,158.518349,157.0,160.0


In [6]:
# Create a new column "team_x win". 1 will indicate that team_x won the match
df['Team_1_Win'] = (df['Team1'] == df['WinningTeam']).astype(int)
df.head()

,ID,City,MatchNumber,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,innings_total,TeamA_batting_average,TeamB_batting_average,TeamA_innings_total,TeamB_innings_total,Team_1_Win
0,1312200,ahmedabad,final,rajasthan royals,gujarat titans,"narendra modi stadium, ahmedabad",rajasthan royals,bat,gujarat titans,130,155.397906,166.437500,130.0,133.0,0
1,1312199,ahmedabad,qualifier 2,royal challengers bangalore,rajasthan royals,"narendra modi stadium, ahmedabad",rajasthan royals,field,rajasthan royals,157,155.707965,155.397906,157.0,161.0,0
2,1312198,kolkata,eliminator,royal challengers bangalore,lucknow super giants,eden gardens,lucknow super giants,field,royal challengers bangalore,207,155.707965,169.866667,207.0,193.0,1
3,1312197,kolkata,qualifier 1,rajasthan royals,gujarat titans,eden gardens,gujarat titans,field,gujarat titans,188,155.397906,166.437500,188.0,191.0,0
4,1304116,mumbai,70,sunrisers hyderabad,punjab kings,wankhede stadium,sunrisers hyderabad,bat,punjab kings,157,155.546053,158.518349,157.0,160.0,0


In [7]:
# Create a new Team1 win toss feature
df['team_1_toss_winner'] = (df['Team1'] == df['TossWinner']).astype(int)

In [8]:
df.MatchNumber.unique()

array(['final', 'qualifier 2', 'eliminator', 'qualifier 1', '70', '69',
       '68', '67', '66', '65', '64', '63', '62', '61', '60', '59', '58',
       '57', '56', '55', '54', '53', '52', '51', '50', '49', '48', '47',
       '46', '45', '44', '43', '42', '41', '40', '39', '38', '37', '36',
       '35', '34', '33', '32', '31', '30', '29', '28', '27', '26', '25',
       '24', '23', '22', '21', '20', '19', '18', '17', '16', '15', '14',
       '13', '12', '11', '10', '9', '8', '7', '6', '5', '4', '3', '2',
       '1', 'qualifier', 'elimination final', '72', '71',
       '3rd place play-off', 'semi final'], dtype=object)

In [9]:
# function
def map_match_number(value):
    if isinstance(value, int) or value.isnumeric():
        return 1
    elif value in ['qualifier 2', 'eliminator', 'qualifier 1', 'qualifier', 'elimination final', '3rd place play-off', 'semi final']:
        return 2
    elif value == 'final':
        return 3
    else:
        return 0

In [10]:
# Apply the function
df['MatchNumber'] = df['MatchNumber'].apply(map_match_number)

# Rename it to something more descriptive 
df.rename(columns={'MatchNumber': 'MatchImportance'}, inplace=True)

In [11]:
df.head(20)

,ID,City,MatchImportance,Team1,Team2,Venue,TossWinner,TossDecision,WinningTeam,innings_total,TeamA_batting_average,TeamB_batting_average,TeamA_innings_total,TeamB_innings_total,Team_1_Win,team_1_toss_winner
0,1312200,ahmedabad,3,rajasthan royals,gujarat titans,"narendra modi stadium, ahmedabad",rajasthan royals,bat,gujarat titans,130,155.397906,166.437500,130.0,133.0,0,1
1,1312199,ahmedabad,2,royal challengers bangalore,rajasthan royals,"narendra modi stadium, ahmedabad",rajasthan royals,field,rajasthan royals,157,155.707965,155.397906,157.0,161.0,0,0
2,1312198,kolkata,2,royal challengers bangalore,lucknow super giants,eden gardens,lucknow super giants,field,royal challengers bangalore,207,155.707965,169.866667,207.0,193.0,1,0
3,1312197,kolkata,2,rajasthan royals,gujarat titans,eden gardens,gujarat titans,field,gujarat titans,188,155.397906,166.437500,188.0,191.0,0,0
4,1304116,mumbai,1,sunrisers hyderabad,punjab kings,wankhede stadium,sunrisers hyderabad,bat,punjab kings,157,155.546053,158.518349,157.0,160.0,0,1
5,1304115,mumbai,1,delhi capitals,mumbai indians,wankhede stadium,mumbai indians,field,mumbai indians,159,153.754464,158.714286,159.0,160.0,0,0
6,1304114,mumbai,1,chennai super kings,rajasthan royals,brabourne stadium,chennai super kings,bat,rajasthan royals,150,160.543269,155.397906,150.0,151.0,0,1
7,1304113,mumbai,1,gujarat titans,royal challengers bangalore,wankhede stadium,gujarat titans,bat,royal challengers bangalore,168,166.437500,155.707965,168.0,170.0,0,1
8,1304112,navi mumbai,1,lucknow super giants,kolkata knight riders,dr dy patil sports academy,lucknow super giants,bat,lucknow super giants,210,169.866667,153.367713,210.0,208.0,1,1
9,1304111,mumbai,1,sunrisers hyderabad,mumbai indians,wankhede stadium,mumbai indians,field,sunrisers hyderabad,193,155.546053,158.714286,193.0,190.0,1,0


In [12]:
# Create your X and y for the model
X = df[['City', 'Venue', 'MatchImportance', 'TossDecision', 'Team1', 'Team2', 'TeamA_batting_average',
       'TeamB_batting_average', 'team_1_toss_winner']]
y = df['TeamA_innings_total']
X.MatchImportance.isna().sum()

0

In [13]:
# Choose which columns to be OneHotEncoded or standardised

categorical_cols = ['City', 'Venue', 'TossDecision', 'Team1', 'Team2']
numerical_cols = ['TeamA_batting_average', 'TeamB_batting_average', 'MatchImportance', 'team_1_toss_winner'] #team 1 toss winner is effectively already encoded

# Create a column transformer to normalise the different types of data
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='if_binary', handle_unknown='ignore'), categorical_cols),
    ('num', MinMaxScaler(), numerical_cols)
])

# Create the pipeline to run the data through
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

X_encoded = pipeline.fit_transform(X)

# Get the column names from the preprocessor
X_encoded = pd.DataFrame.sparse.from_spmatrix(X_encoded, columns=preprocessor.get_feature_names_out())


X_encoded

,cat__City_abu dhabi,cat__City_ahmedabad,cat__City_bangalore,cat__City_bengaluru,cat__City_bloemfontein,cat__City_cape town,cat__City_centurion,cat__City_chandigarh,cat__City_chennai,cat__City_cuttack,...,cat__Team2_pune warriors,cat__Team2_punjab kings,cat__Team2_rajasthan royals,cat__Team2_rising pune supergiant,cat__Team2_royal challengers bangalore,cat__Team2_sunrisers hyderabad,num__TeamA_batting_average,num__TeamB_batting_average,num__MatchImportance,num__team_1_toss_winner
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.575459,0.899382,1.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.584557,0.575459,0.5,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.584557,1.000000,0.5,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.575459,0.899382,0.5,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.579806,0.667019,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.500405,0.515889,0.0,0.0
944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.672768,0.584557,0.0,1.0
945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.575459,0.527237,0.0,0.0
946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.726434,0.667019,0.0,0.0


In [14]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Initialize the Linear Regression model
model = GradientBoostingRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Mean Squared Error: 730.27
Root Mean Squared Error: 27.02
R-squared (R2) Score: -0.04


In [15]:
# Initialize the GradientBoostingRegressor model
model = GradientBoostingRegressor()

# Define the hyperparameter grid for the random search
param_grid = {
    'n_estimators': np.arange(50, 251, 10),        
    'learning_rate': [0.01, 0.1, 0.2, 0.3, 0.5],  
    'max_depth': np.arange(3, 11),                
    'min_samples_split': np.arange(2, 11),        
    'min_samples_leaf': np.arange(1, 11),        
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0]      
}

# Initialize the RandomizedSearchCV object
random_search = RandomizedSearchCV(
    model, 
    param_distributions=param_grid, 
    n_iter=50, 
    scoring='r2', 
    cv=5, 
    random_state=42, 
    n_jobs=-1
)

# Fit the random search to the training data
random_search.fit(X_train, y_train)

# Get the best model from the random search
best_model = random_search.best_estimator_

# Make predictions on the test data using the best model
y_pred = best_model.predict(X_test)

# Evaluate the best model
mse = mean_squared_error(y_test, y_pred)
rmse = mse**0.5
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Best Model Hyperparameters:")
print(random_search.best_params_)
print(f"Mean Squared Error: {mse:.2f}")
print(f"Root Mean Squared Error: {rmse:.2f}")
print(f"R-squared (R2) Score: {r2:.2f}")

Best Model Hyperparameters:
{'subsample': 0.6, 'n_estimators': 60, 'min_samples_split': 3, 'min_samples_leaf': 5, 'max_depth': 3, 'learning_rate': 0.01}
Mean Squared Error: 690.84
Root Mean Squared Error: 26.28
R-squared (R2) Score: 0.02


In [16]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# CREATE A SEPARATE X and y to try a XGBoost model
X2 = df[['City', 'Venue', 'MatchImportance', 'TossDecision', 'Team1', 'Team2', 'TeamA_batting_average',
       'TeamB_batting_average', 'team_1_toss_winner']]
y2 = df['Team_1_Win']

# encode the new X2 data
# Choose which columns to be OneHotEncoded or LabelEncoded
categorical_cols = ['City', 'Venue', 'TossDecision', 'Team1', 'Team2' ]
numerical_cols = ['TeamA_batting_average', 'TeamB_batting_average', 'team_1_toss_winner', 'MatchImportance']

# Create a column transformer to normalise the different types of data
preprocessor2 = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='if_binary', handle_unknown='ignore'), categorical_cols),
    ('num', RobustScaler(), numerical_cols)
])

# Create the pipeline to run the data through
pipeline3 = Pipeline([
    ('preprocessor', preprocessor2)
])

X2_encoded = pipeline3.fit_transform(X)

# Get the column names from the preprocessor
X2_encoded = pd.DataFrame.sparse.from_spmatrix(X2_encoded, columns=preprocessor2.get_feature_names_out())
X2_encoded



# Try with a simple Logistic Regression
# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_encoded, y2, test_size=0.2, random_state=42, shuffle=True)

# Initialize and train the logistic regression model
model = LogisticRegression(penalty='l2', C=5, random_state=42)
model.fit(X2_train, y2_train)

# Make predictions on the testing set
y2_pred = model.predict(X2_test)

# Evaluate the model
accuracy = round(accuracy_score(y2_test, y2_pred), 4)

print(f'Accuracy: {accuracy}')

Accuracy: 0.5053


In [17]:
# Do a random grid search to optimize the model

# Define the hyperparameter grid you want to search over
param_grid = {
    'penalty': ['l1', 'l2'],  
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  
    'solver': ['liblinear', 'saga'] 
}

# Create the Logistic Regression model
logistic_regression = LogisticRegression()

# Create the gridsearch
grid_search = GridSearchCV(logistic_regression, 
                           param_grid, 
                           cv=5,
                           n_jobs=-1,
                           scoring='accuracy')

# Fit the random search to the data
grid_search.fit(X2_train, y2_train)

# Get the best hyperparameters from the random search
best_params = grid_search.best_params_

# Get the best estimator (model) from the random search
best_model = grid_search.best_estimator_

# Make predictions on the testing set using the best model
y2_pred = best_model.predict(X2_test)

# Evaluate the best model
accuracy = round(accuracy_score(y2_test, y2_pred), 4)

print(f'Best Hyperparameters: {best_params}')
print(f'Best Model Accuracy: {accuracy}')


/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag

Best Hyperparameters: {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Model Accuracy: 0.5053


/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [18]:
import xgboost as xgb

# Initialize and train the model
# Create an XGBoost classifier
clf = xgb.XGBClassifier(
    objective='binary:logistic',  
    n_estimators=1000,            
    max_depth=10,                 
    learning_rate=0.01            
)
clf.fit(X2_train, y2_train)

# Make predictions on the testing set
y2_pred = clf.predict(X2_test)

# Evaluate the model
accuracy = round(accuracy_score(y2_test, y2_pred), 2)

print(f'Accuracy: {accuracy}')

Accuracy: 0.52


In [ ]:
from scipy.stats import uniform, randint
# RandomSearch the XGBoost

# Define hyperparameter values
param_dist = {
    'n_estimators': randint(100, 2001),  
    'max_depth': randint(3, 10),  
    'learning_rate': uniform(0.01, 1),  
    'subsample': uniform(0.2, 0.9),  
    'colsample_bytree': uniform(0.2, 0.9)}

# Create the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', random_state=42)

# Create the RandomizedSearchCV object with 5-fold cross-validation
random_search = RandomizedSearchCV(xgb_classifier, 
                                   param_distributions=param_dist, 
                                   n_iter=100, 
                                   cv=5,
                                   n_jobs=-1,
                                   scoring='accuracy', 
                                   random_state=42)

# Fit the random search to the data
random_search.fit(X2_train, y2_train)

# Get the best hyperparameters from the random search
best_params = random_search.best_params_

# Get the best estimator (model) from the random search
best_model = random_search.best_estimator_

# Make predictions on the testing set using the best model
y2_pred = best_model.predict(X2_test)

# Evaluate the best model
accuracy = round(accuracy_score(y2_test, y2_pred), 2)

print(f'Best Hyperparameters: {best_params}')
print(f'Best Model Accuracy: {accuracy}')

In [20]:
df.columns

Index(['ID', 'City', 'MatchImportance', 'Team1', 'Team2', 'Venue',
       'TossWinner', 'TossDecision', 'WinningTeam', 'innings_total',
       'TeamA_batting_average', 'TeamB_batting_average', 'TeamA_innings_total',
       'TeamB_innings_total', 'Team_1_Win', 'team_1_toss_winner'],
      dtype='object')

In [21]:
df.Venue.unique()

array(['narendra modi stadium, ahmedabad', 'eden gardens',
       'wankhede stadium', 'brabourne stadium',
       'dr dy patil sports academy',
       'maharashtra cricket association stadium',
       'dubai international cricket stadium', 'sharjah cricket stadium',
       'sheikh zayed stadium', 'arun jaitley stadium',
       'ma chidambaram stadium', 'rajiv gandhi international stadium',
       'dr. y.s. rajasekhara reddy aca-vdca cricket stadium',
       'punjab cricket association is bindra stadium',
       'm.chinnaswamy stadium', 'sawai mansingh stadium',
       'holkar cricket stadium', 'feroz shah kotla', 'green park',
       'saurashtra cricket association stadium',
       'shaheed veer narayan singh international stadium',
       'jsca international stadium complex',
       'sardar patel stadium, motera', 'barabati stadium',
       'subrata roy sahara stadium',
       'himachal pradesh cricket association stadium', 'nehru stadium',
       'vidarbha cricket association stadium